In [471]:
import warnings
warnings.filterwarnings('ignore')

import pandas as pd
import numpy as np
import geopandas as gpd

from sklearn.model_selection import train_test_split
from sklearn.metrics import mean_absolute_error
from sklearn.linear_model import LinearRegression
from geopy.geocoders import Nominatim
import overpass
import geocoder
from tqdm import tqdm_notebook
from sklearn.preprocessing import LabelEncoder

In [472]:
import overpy
from sklearn.preprocessing import StandardScaler

### Задача:

Прогноз продаж одной из популярных моделей [фичерфонов](https://ru.wikipedia.org/wiki/%D0%A4%D0%B8%D1%87%D0%B5%D1%80%D1%84%D0%BE%D0%BD) (на картинке ниже пример похожего устройства) в салонах МегаФона
![](https://39.img.avito.st/640x480/8468720439.jpg)

### Исходные данные:

Датасет содержит следующие поля:

1. `point_id` - Индентификатор салона
2. `lon` - Долгота точки
3. `lat` - Широта точки
4. `target` - Значение таргета, усредненное за несколько месяцев и отнормированное

### Требования к решению и советы:

Ниже приведен список из нескольких важных пунктов, необходимых для решения задания. Выполнение каждого из пунктов влияет на итоговую оценку. Вы можете выполнить каждый из пунктов разными способами, самым лучшим будет считаться вариант, когда всё получение и обработка данных будут реализованы на Питоне (пример: вы можете скачать данные из OSM через интерфейс на сайте overpass-turbo или с помощью библиотек `overpass`/`requests`. Оба варианта будут зачтены, но больше баллов можно заработать во втором случае)



1. Салоны расположены в нескольких разных городах, вам необходимо **определить город для каждого салона** (это понадобится во многих частях задания). К этому есть разные подходы. Вы можете провести [обратное геокодирование](https://en.wikipedia.org/wiki/Reverse_geocoding) с помощью геокодера [Nominatim](https://nominatim.org/), доступного через библиотеку `geopy` примерно вот так:
```python
from geopy.geocoders import Nominatim
geolocator = Nominatim(user_agent="specify_your_app_name_here")
location = geolocator.reverse("52.509669, 13.376294")
print(location.address)
```
В таком случае, вам придется обрабатывать полученную строку адреса, чтобы извлечь название города. Также вы можете скачать из OSM или найти в любом другом источнике границы административно территориальных границ России и пересечь с ними датасет с помощью `geopandas.sjoin` (этот вариант более надежный, но нужно будет разобраться с тем, как устроены границы АТД в OSM, обратите внимание на [этот тег](https://wiki.openstreetmap.org/wiki/Key:admin_level))


2. **Используйте данные OSM**: подумайте, какие объекты могут влиять на продажи фичерфонов. Гипотеза: такие телефоны покупают люди, приезжающие в город или страну ненадолго, чтобы вставить туда отдельную симкарту для роуминга. Можно попробовать использовать местоположения железнодорожных вокзалов (изучите [этот тег](https://wiki.openstreetmap.org/wiki/Tag:railway%3Dstation)). Необходимо использовать хотя бы 5 разных типов объектов из OSM. Скорее всего, вам придется качать данные OSM отдельно для разных городов (см. пример для Нью-Йорка из лекции)


3. **Используйте разные способы генерации признаков**: описать положение салона МегаФона относительно станций метро можно разными способами - найти ***расстояние до ближайшей станции***, или же посчитать, сколько станций попадает в ***500 метровую буферную зону*** вокруг салона. Такие признаки будут нести разную информацию. Так же попробуйте поэкспериментировать с размерами буферных зон (представьте, что значат в реальности радиусы 100, 500, 1000 метров). Попробуйте посчитать расстояние до центра города, до других объектов.

4. **Сделайте визуализации**: постройте 2-3 карты для какого нибудь из городов - как распределен в пространстве таргет, где находятся объекты, полученные вами из OSM. Можете использовать любой инструмент - обычный `plot()`, `folium`, `keplergl`. Если выберете Кеплер, обязательно сохраните в файл конфиг карты, чтобы ее можно было воспроизвести. Сделать это можно вот так:

```python
import json
json_data = kepler_map.config
with open('kepler_config.json', 'w') as outfile:
    json.dump(json_data, outfile)
```
5. Задание не ограничено приведенными выше пунктами, попробуйте нагенерировать интересных признаков, найти в интернете дополнительные данные (в таком случае в комментарии к коду укажите ссылку на ресурс, откуда взяли данные)



6. Это довольно сложная задача - датасет очень маленький, данные по своей природе довольно случайны. Поэтому место и скор на Kaggle не будут играть решающую роль в оценке, но позволят заработать дополнительные баллы

In [473]:
geolocator = Nominatim(user_agent="Kindenko",scheme = 'https', timeout=100)
location = geolocator.reverse("55.848630, 37.590776")
print(location.raw['address']['state']) 
#Имя только Kindenko!!

Москва


In [25]:
#from geopy.geocoders import Nominatim
geolocator = Nominatim(user_agent="Kindenko")
location = geolocator.reverse("52.509669, 13.376294")
print(location.address)

Potsdamer Platz, Fontaneplatz, Tiergarten, Mitte, Berlin, Rixdorf, Berlin, 10785, Deutschland


In [20]:
#from OSMPythonTools.overpass import Overpass
#from OSMPythonTools.nominatim import Nominatim

In [474]:
import requests
import json
from keplergl import KeplerGl
from geopy import distance

In [475]:
import utm

In [476]:
def find_object(data):
    final_data = {}
    for i in data:
        overpass_url = "http://overpass-api.de/api/interpreter"
        overpass_query = f"""
        [out:json];
        area[name="{i}"]->.searchArea;
        (
        node["railway"="station"](area.searchArea);
        way["railway"="station"](area.searchArea);
        relation["railway"="station"](area.searchArea);
        );

        out body;
        >;
        out skel qt;
        """
        response = requests.get(overpass_url, params={'data': overpass_query})
        data = response.json()
        final_data[i] = data
    return final_data

In [477]:
def find_coordinate_train_object(data):
    coordinate = {}
    for i in data:
        coordinate[i] = {}
        for j in range(len(data[i]['elements'])):
            try:
                coo = []
                coo.append(data[i]['elements'][j]['lat'])
                coo.append(data[i]['elements'][j]['lon'])
                coordinate[i].update({j:coo})
            except:
                continue
    return coordinate

### Read data

In [478]:
train = pd.read_csv('mf_geo_train.csv')
test = pd.read_csv('mf_geo_test.csv')

# Города в трейн

In [479]:
train.head()

,point_id,lon,lat,target
0,ommNZCUV,37.590776,55.848630,-0.348157
1,nMe2LHPb,37.784210,55.750271,1.294206
2,ZgodVRqB,39.635721,47.213330,-1.039679
3,0t2jNYdz,37.704570,55.782020,-1.169339
4,U27W4QJ7,37.643983,55.730188,-0.088837


In [480]:
coo_train = train.iloc[:, 1:3]

In [481]:
city_train = pd.DataFrame(columns = ['city'])

In [482]:
for i in range(coo_train.shape[0]):
        coordinate = str(coo_train.values[i][1]) + ',' + str(coo_train.values[i][0])
        geolocator = Nominatim(user_agent='Kindenko')
        location = geolocator.reverse(coordinate)
        try:
            if 'район' in location.raw['address']['city']:
                city_train.loc[i, 'city'] = location.raw['address']['state']
            else:
                city_train.loc[i, 'city'] = location.raw['address']['city']
        except:
            city_train.loc[i, 'city'] = location.raw['address']['state']       

In [483]:
city_train['city'].unique()

array(['Москва', 'Ростов-на-Дону', 'Красноярск', 'Санкт-Петербург', 'Уфа',
       'городской округ Казань', 'Екатеринбург', 'Нижний Новгород',
       'Колпино', 'Новосибирск', 'городской округ Новосибирск',
       'городской округ Самара', 'Самара', 'Казань', 'Зеленоград',
       'Балашиха', 'Пушкин'], dtype=object)

In [484]:
city_train['city'].value_counts()

Москва                         157
Санкт-Петербург                 78
городской округ Самара          22
Екатеринбург                    22
Ростов-на-Дону                  21
Нижний Новгород                 21
Красноярск                      20
Уфа                             19
городской округ Казань          17
городской округ Новосибирск     14
Новосибирск                     12
Казань                           8
Самара                           5
Колпино                          4
Зеленоград                       3
Балашиха                         1
Пушкин                           1
Name: city, dtype: int64

In [485]:
city_train

,city
0,Москва
1,Москва
2,Ростов-на-Дону
3,Москва
4,Москва
...,...
420,Москва
421,Москва
422,Москва
423,Москва


In [486]:
train = train.join(city_train)

# Присвоим выбросам нормальные города

In [487]:
for i in range(len(train['city'])):
    if train['city'][i] == 'городской округ Казань':
        train['city'][i] = 'Казань'
    if train['city'][i] == 'городской округ Самара':
        train['city'][i] = 'Самара'
    if train['city'][i] == 'городской округ Новосибирск':
        train['city'][i] = 'Новосибирск'
    if train['city'][i] == 'Колпино':
        train['city'][i] = 'Санкт-Петербург'
    if train['city'][i] == 'Колпино':
        train['city'][i] = 'Санкт-Петербург'
    if train['city'][i] == 'Зеленоград':
        train['city'][i] = 'Москва'
    if train['city'][i] == 'Балашиха':
        train['city'][i] = 'Москва'
    if train['city'][i] == 'Пушкин':
        train['city'][i] = 'Санкт-Петербург'

# Какие жд объекты в радиусе 100м 

In [488]:
def find_object(data):
    final_data = {}
    for i in data:
        overpass_url = "http://overpass-api.de/api/interpreter"
        overpass_query = f"""
        [out:json];
        area[name="{i}"]->.searchArea;
        (
        node["railway"="station"](area.searchArea);
        way["railway"="station"](area.searchArea);
        relation["railway"="station"](area.searchArea);
        );

        out body;
        >;
        out skel qt;
        """
        response = requests.get(overpass_url, params={'data': overpass_query})
        data = response.json()
        final_data[i] = data
    return final_data

In [489]:
def find_coordinate_train_object(data):
    coordinate = {}
    for i in data:
        coordinate[i] = {}
        for j in range(len(data[i]['elements'])):
            try:
                coo = []
                coo.append(data[i]['elements'][j]['lat'])
                coo.append(data[i]['elements'][j]['lon'])
                coordinate[i].update({j:coo})
            except:
                continue
    return coordinate

In [490]:
name_city = ('Москва','Санкт-Петербург','Самара','Новосибирск','Казань','Екатеринбург','Нижний Новгород','Ростов-на-Дону','Красноярск','Уфа')

In [491]:
train_object = find_object(name_city)

In [493]:
coo_train_object = find_coordinate_train_object(train_object)

In [502]:
def train_in_radius(coordinate, data):
    radius = pd.DataFrame(0, index=range(len(data)),columns = ['radius_1000'])
    for i in range(len(data['city'])):
        for j in coordinate:
            if data['city'][i] == j:
                for k in range(len(coordinate[j])):
                    try:
                        coo1 = []
                        coo2 = []
                        coo1.append(data['lat'][i])
                        coo1.append(data['lon'][i])
                        coo2.append(coordinate[j][k][0])
                        coo2.append(coordinate[j][k][1]) 
                        dis = distance.geodesic(coo1,coo2).m
                        if dis < 1000:
                                radius.loc[i] = 1  
                    except:
                        continue
    return radius

In [497]:
%%time
radius_100 = train_in_radius(coo_train_object, train)

Wall time: 12.7 s


In [499]:
radius_250 = train_in_radius(coo_train_object, train)

In [501]:
radius_500 = train_in_radius(coo_train_object, train)

In [503]:
radius_1000 = train_in_radius(coo_train_object, train)

In [504]:
train = train.join(radius_100)
train = train.join(radius_250)
train = train.join(radius_500)
train = train.join(radius_1000)

In [505]:
train[(train['city'] == 'Москва') & (train['radius_1000'] == 1)]

,point_id,lon,lat,target,city,radius_100,radius_250,radius_500,radius_1000
0,ommNZCUV,37.590776,55.848630,-0.348157,Москва,0,1,1,1
1,nMe2LHPb,37.784210,55.750271,1.294206,Москва,0,1,1,1
3,0t2jNYdz,37.704570,55.782020,-1.169339,Москва,1,1,1,1
4,U27W4QJ7,37.643983,55.730188,-0.088837,Москва,0,1,1,1
7,CO76tdVs,37.444304,55.850511,0.213704,Москва,0,0,1,1
...,...,...,...,...,...,...,...,...,...
420,AA8hN7bJ,37.628765,55.740664,0.213704,Москва,1,1,1,1
421,uNw6t6xk,37.586825,55.794233,-0.002397,Москва,0,1,1,1
422,Ap42ei8k,37.678790,55.772910,-0.910019,Москва,0,1,1,1
423,rn9A8r62,37.752130,55.619640,-0.326547,Москва,0,0,0,1


# Минимальное расстояние от салона до ЖД 

In [506]:
def dis_for_train(coordinate, data):
    radius = pd.DataFrame(0, index=range(len(data)),columns = ['dis_for_railway'])
    for i in range(len(data['city'])):
        for j in coordinate:
            if data['city'][i] == j:
                for k in range(len(coordinate[j])):
                    try:
                        coo1 = []
                        coo2 = []
                        coo1.append(data['lat'][i])
                        coo1.append(data['lon'][i])
                        coo2.append(coordinate[j][k][0])
                        coo2.append(coordinate[j][k][1]) 
                        dis = distance.geodesic(coo1,coo2).m
                        if 
                        radius.loc[i] = dis  
                    except:
                        continue
    return radius

In [507]:
rasstoyanie = dis_for_train(coo_train_object,train)

In [509]:
coo_train_object

{'Москва': {0: [55.7788343, 37.6537207],
  1: [55.7268684, 37.449885],
  2: [55.8871767, 37.66155],
  3: [55.8696254, 37.6641842],
  4: [55.7577737, 37.6621837],
  5: [55.7885334, 37.7510006],
  6: [55.7833072, 37.7213614],
  7: [55.7742475, 37.659914],
  8: [55.7932703, 37.6321318],
  9: [55.7394565, 37.4834225],
  10: [55.7436954, 37.4978443],
  11: [55.7461002, 37.5148876],
  12: [55.7400295, 37.534383],
  13: [55.7388362, 37.5483675],
  14: [55.7817059, 37.7038046],
  15: [55.7730735, 37.6805829],
  16: [55.7566597, 37.6218635],
  17: [55.7521035, 37.6069401],
  18: [55.7407773, 37.4168044],
  19: [55.7359311, 37.4671952],
  20: [55.7607707, 37.6261868],
  21: [55.740465, 37.6518996],
  22: [55.7318655, 37.6658787],
  23: [55.7252622, 37.6868197],
  24: [55.7088457, 37.7315088],
  25: [55.7056021, 37.7657844],
  26: [55.7156444, 37.8179229],
  27: [55.7171259, 37.793436],
  28: [55.7518652, 37.6007518],
  29: [55.7500874, 37.5413666],
  30: [55.7524027, 37.6086825],
  31: [55.74370

In [508]:
rasstoyanie

,dis_for_railway
0,4021.297602
1,12407.220402
2,10794.541924
3,6472.024626
4,11072.776929
...,...
420,9960.694857
421,5386.133473
422,6654.551236
423,24323.745327


# Найдем расстояние от салона до центра города

In [399]:
#координаты центра всех городов
def find_city_center(data):
    final_data = {}
    for i in data:
        overpass_url = "http://overpass-api.de/api/interpreter"
        overpass_query = f"""
        [out:json];
        area[name="{i}"]->.searchArea;
        (
        node["place"="city"]["name:ru"="{i}"](area.searchArea);
        );

        out body;
        >;
        out skel qt;
        """
        response = requests.get(overpass_url, params={'data': overpass_query})
        data = response.json()
        final_data[i] = data
    return final_data

In [400]:
coo_city_centre = find_city_center(name_city)

In [401]:
def find_distance_to_centre(coordinate, data):
    distance_to_center = pd.DataFrame(0, index=range(len(data)),columns = ['distance_to_centre'])
    for i in range(len(data['city'])):
        for j in coordinate:
            if data['city'][i] == j:
                    try:
                        coo1 = []
                        coo2 = []
                        coo1.append(data['lat'][i])
                        coo1.append(data['lon'][i])
                        coo2.append(coordinate[j]['elements'][0]['lat'])
                        coo2.append(coordinate[j]['elements'][0]['lon']) 
                        dis = distance.geodesic(coo1,coo2).m
                        distance_to_center.loc[i] = dis  
                    except:
                        continue
    return distance_to_center

In [402]:
distance = find_distance_to_centre(coo_city_centre,train)

In [403]:
distance.loc[distance['distance_to_centre'] < 1000]

,distance_to_centre
10,433.035301
25,727.961763
40,271.463960
61,939.171254
65,751.889269
68,442.362554
81,913.618888
179,36.518288
230,747.169751
231,161.004516


In [404]:
train = train.join(distance)

In [405]:
train

,point_id,lon,lat,target,city,radius_100,radius_250,radius_500,radius_1000,distance_to_centre
0,ommNZCUV,37.590776,55.848630,-0.348157,Москва,0,1,1,1,11059.296189
1,nMe2LHPb,37.784210,55.750271,1.294206,Москва,0,1,1,1,10468.820770
2,ZgodVRqB,39.635721,47.213330,-1.039679,Ростов-на-Дону,0,0,0,0,5803.464445
3,0t2jNYdz,37.704570,55.782020,-1.169339,Москва,1,1,1,1,6498.538254
4,U27W4QJ7,37.643983,55.730188,-0.088837,Москва,0,1,1,1,2802.731790
...,...,...,...,...,...,...,...,...,...,...
420,AA8hN7bJ,37.628765,55.740664,0.213704,Москва,1,1,1,1,1298.914111
421,uNw6t6xk,37.586825,55.794233,-0.002397,Москва,0,1,1,1,5241.342723
422,Ap42ei8k,37.678790,55.772910,-0.910019,Москва,0,1,1,1,4589.311225
423,rn9A8r62,37.752130,55.619640,-0.326547,Москва,0,0,0,1,16846.622989


# Найдем количество магазинов в окресности салона.

In [406]:
#Магазины в окрестности
def find_object_shop(points,shift):
    final_data = list()
    excepted = list()
    count = 0
    for i in points:
        low_edge = radian_shift(i[0], i[1], North_shift = -shift, East_shift= -shift)
        high_edge = radian_shift(i[0], i[1], North_shift = shift, East_shift= shift)
        overpass_url = "http://overpass-api.de/api/interpreter"
        overpass_query = f"""
        [out:json];
        (
        node["shop"]({low_edge[0]},{low_edge[1]},{high_edge[0]},{high_edge[1]});
        );

        out body;
        >;
        out skel qt;
        """
        try:
          response = requests.get(overpass_url, params={'data': overpass_query})
          data = response.json()
          count += 1
          #print("Processed",i)
        except:
          print("Excepted points: ",i," Row: ",count)
          data = find_object_shop([i],shift)
          print("Exception resolved: ",i," Row: ",count)
          count += 1
        final_data.append(data)
    return final_data

In [407]:
def radian_shift(lat, lon, North_shift = 100, East_shift= 100):
    point = utm.from_latlon(lat,lon)
    lat_fin = point[1] + North_shift
    lon_fin = point[0] + East_shift
    point = utm.to_latlon(lon_fin, lat_fin, point[2], point[3])
    return [point[0],point[1]]

In [408]:
def find_coordinate_shop_object(data):
    coordinate = []
    for i in range(len(data)):
        coo = []
        coo.append(data['lat'][i])
        coo.append(data['lon'][i])
        coordinate.append(coo)
    return coordinate

In [409]:
coordinate_salon = find_coordinate_shop_object(coo_train)

In [410]:
%%time
shops = find_object_shop(coordinate_salon, 200)

Wall time: 10min 1s


In [308]:
for i in range(len(shops)):
    count_shops['count_shops'][i] = len(shops[i]['elements'])

In [411]:
train = train.join(count_shops)

In [412]:
train

,point_id,lon,lat,target,city,radius_100,radius_250,radius_500,radius_1000,distance_to_centre,count_shops
0,ommNZCUV,37.590776,55.848630,-0.348157,Москва,0,1,1,1,11059.296189,9
1,nMe2LHPb,37.784210,55.750271,1.294206,Москва,0,1,1,1,10468.820770,40
2,ZgodVRqB,39.635721,47.213330,-1.039679,Ростов-на-Дону,0,0,0,0,5803.464445,1
3,0t2jNYdz,37.704570,55.782020,-1.169339,Москва,1,1,1,1,6498.538254,50
4,U27W4QJ7,37.643983,55.730188,-0.088837,Москва,0,1,1,1,2802.731790,29
...,...,...,...,...,...,...,...,...,...,...,...
420,AA8hN7bJ,37.628765,55.740664,0.213704,Москва,1,1,1,1,1298.914111,76
421,uNw6t6xk,37.586825,55.794233,-0.002397,Москва,0,1,1,1,5241.342723,51
422,Ap42ei8k,37.678790,55.772910,-0.910019,Москва,0,1,1,1,4589.311225,69
423,rn9A8r62,37.752130,55.619640,-0.326547,Москва,0,0,0,1,16846.622989,14


# Визуализация

In [462]:
map_2 = KeplerGl(height=400, data={"data_1": train})
map_2

User Guide: https://docs.kepler.gl/docs/keplergl-jupyter


KeplerGl(data={'data_1':      point_id        lon        lat    target  radius_100  radius_250  \
0    ommNZCU…

In [463]:
json_data = map_2.config
with open('kepler_config.json', 'w') as outfile:
    json.dump(json_data, outfile)

In [464]:
train_moscow = train.loc[train['city'] == 'Москва']

In [466]:
map_3 = KeplerGl(height=400, data={"data_2": train_moscow})
map_3

User Guide: https://docs.kepler.gl/docs/keplergl-jupyter


KeplerGl(data={'data_2':      point_id        lon        lat    target  radius_100  radius_250  \
0    ommNZCU…

In [467]:
json_data = map_3.config
with open('kepler_config.json', 'w') as outfile:
    json.dump(json_data, outfile)

In [468]:
train_spb = train.loc[train['city'] == 'Санкт-Петербург']

In [469]:
map_4 = KeplerGl(height=400, data={"data_2": train_spb})
map_4

User Guide: https://docs.kepler.gl/docs/keplergl-jupyter


KeplerGl(data={'data_2':      point_id        lon        lat    target  radius_100  radius_250  \
6    O3tOF6u…

In [470]:
json_data = map_4.config
with open('kepler_config.json', 'w') as outfile:
    json.dump(json_data, outfile)

# Преобразовываем данные и стандартизируем

In [425]:
labelencoder = LabelEncoder()

In [426]:
scaler = StandardScaler()

In [427]:
train.iloc[:, 4] = labelencoder.fit_transform(train.iloc[:, 4])

In [450]:
train.iloc[:, 9] = scaler.fit_transform(train.iloc[:, 9].values.reshape(-1,1))

In [451]:
train.head()

,point_id,lon,lat,target,city,radius_100,radius_250,radius_500,radius_1000,distance_to_centre,count_shops
0,ommNZCUV,37.590776,55.848630,-0.348157,-0.576948,-0.365148,1.261532,0.983663,0.761528,0.324956,-0.699850
1,nMe2LHPb,37.784210,55.750271,1.294206,-0.576948,-0.365148,1.261532,0.983663,0.761528,0.233303,0.270731
2,ZgodVRqB,39.635721,47.213330,-1.039679,0.577854,-0.365148,-0.792687,-1.016608,-1.313148,-0.490852,-0.950323
3,0t2jNYdz,37.704570,55.782020,-1.169339,-0.576948,2.738613,1.261532,0.983663,0.761528,-0.382963,0.583822
4,U27W4QJ7,37.643983,55.730188,-0.088837,-0.576948,-0.365148,1.261532,0.983663,0.761528,-0.956624,-0.073668


### Fit model

In [315]:
X_train, X_valid, y_train, y_valid = train_test_split(train.drop('target', axis=1), train[['target']])
model = LinearRegression().fit(X_train.drop('point_id', axis=1), y_train)

In [316]:
mean_absolute_error(y_valid, model.predict(X_valid.drop('point_id', axis=1)))

0.7103871528749965

### Make submission

In [452]:
submission = pd.read_csv('sample_submission.csv')
submission['target'] = model.predict(X_valid.drop('point_id', axis=1))
submission.to_csv('my_submission_06.csv', index=False)